In [1]:
import gzip
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_absolute_error
from collections import defaultdict
from itertools import product, combinations
from operator import itemgetter
import random

In [2]:
# These are the functions you need to read in the data. You need to have the datafiles in a data folder that is in
# the directory you are working in
#LINK IM USING FOR K-NN https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/


def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getData(name, attributes):
    i = 0
    dic = {}
    path = 'data/reviews_%s_5.json.gz' % name
    for line in parse(path):
        filtered = {}
        for k in line.keys():
            if k in attributes:
                filtered[k] = line[k]
            else:
                continue
        
        dic[i] = filtered
        i += 1
    return dic

In [3]:
# These are one of the three names you can choose to read the dataset from
names = ['Digital_Music', 'Kindle_Store', 'Video_Games']

# These are all the possible attributes that our datapoints can have. Lots of them are not that useful for us. So during
# the reading of the datafile you have to specify which of these attributes you want to include. For our purposes,
# only reviewerID, asin (product ID) and overall will be helpful. But I included them all, just in case.
attributes = ['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 
              'unixReviewTime', 'reviewTime']

In [4]:
# You read in the data for example like this:
data = getData('Digital_Music', ['reviewerID', 'asin', 'overall'])

In [5]:
# This is what our data will look like. You have a dictionary with integers as keys [0,1,2,3,4...], and as values one 
# datapoint. Each datapoint in itself is a dictionary, with as keys the attribute, and as value the value of that attribute.
for i in range(5):
    print(data[i])

{'reviewerID': 'A3EBHHCZO6V2A4', 'asin': '5555991584', 'overall': 5.0}
{'reviewerID': 'AZPWAXJG9OJXV', 'asin': '5555991584', 'overall': 5.0}
{'reviewerID': 'A38IRL0X2T4DPF', 'asin': '5555991584', 'overall': 5.0}
{'reviewerID': 'A22IK3I6U76GX0', 'asin': '5555991584', 'overall': 5.0}
{'reviewerID': 'A1AISPOIIHTHXX', 'asin': '5555991584', 'overall': 4.0}


In [5]:
def split_data(data, test_ratio=0.1):
    """This function splits the data into a train and test set, randomly extracting users and all their ratings to put
    them in the test set. The test ratio can be specified as a number between 0 and 1, depending on how much percent of
    the data should be in the test set."""
    reviewers = set()
    for i in data.items():
        reviewers.add(i[1]['reviewerID'])
    
    l = len(reviewers)
    test_reviewers = set(random.sample(reviewers, int(l*test_ratio)))
    train_reviewers = reviewers - test_reviewers
    
    train_data = {k:v for k,v in data.items() if v['reviewerID'] in train_reviewers}
    test_data = {k:v for k,v in data.items() if v['reviewerID'] in test_reviewers}
    
    
    tr = len(train_data)
    te = len(test_data)
    print('There are %s train reviews' %tr)
    print('There are %s test reviews' %te)
    
    return train_data, test_data

In [6]:
train_data, test_data = split_data(data)

There are 57236 train reviews
There are 7470 test reviews


In [7]:
ds_train = pd.DataFrame.from_dict(train_data, orient='index')
ds_test = pd.DataFrame.from_dict(test_data, orient='index')

print('Number of colums in training Dataframe : ', len(ds_train.columns))
print('Number of rows in training Dataframe : ', len(ds_train.index))
ds_train.head()

Number of colums in training Dataframe :  3
Number of rows in training Dataframe :  57236


,reviewerID,asin,overall
0,A3EBHHCZO6V2A4,5555991584,5.0
1,AZPWAXJG9OJXV,5555991584,5.0
2,A38IRL0X2T4DPF,5555991584,5.0
3,A22IK3I6U76GX0,5555991584,5.0
4,A1AISPOIIHTHXX,5555991584,4.0


In [8]:
X_train = ds_train.iloc[:,:].values
X_test = ds_test.iloc[:,:].values

print(X_train[:6])
print(X_test[:6])

[['A3EBHHCZO6V2A4' '5555991584' 5.0]
 ['AZPWAXJG9OJXV' '5555991584' 5.0]
 ['A38IRL0X2T4DPF' '5555991584' 5.0]
 ['A22IK3I6U76GX0' '5555991584' 5.0]
 ['A1AISPOIIHTHXX' '5555991584' 4.0]
 ['A2P49WD75WHAG5' '5555991584' 5.0]]
[['A24N1BAS3CU27H' '5555991584' 5.0]
 ['A200C7YQJ45LRR' 'B0000000ZW' 4.0]
 ['A8MWQY1R5YE3M' 'B00000016T' 5.0]
 ['A4VVYYB68NL4Z' 'B00000016T' 4.0]
 ['A22N03OBDDVSEB' 'B00000016T' 5.0]
 ['A3464G00K8ZYD1' 'B00000016T' 5.0]]


In [9]:
# These are the three similarity measures that we make use of. Cosine looks at the angle between two datapoints, euclidean
# at their distance in space, and pearson correlation is a measure of their linear dependence.

def cosine_similarity(p,q):
    d = sum(pi * qi for pi,qi in zip(p, q))
    mag_p = math.sqrt(sum([pi**2 for pi in p]))
    mag_q = math.sqrt(sum([qi**2 for qi in q]))
    sim = d / ( mag_p * mag_q)
    return sim

def euclidean_similarity(p, q):
    dist = math.sqrt(sum((pi-qi)**2 for pi,qi in zip(p, q)))
    sim = 1 / (1+dist)
    return sim    

def pearson_correlation(p,q):
    # this code does not scale well to large datasets. In the following, we rely on 
    # scipy.spatial.distance.correlation() to compute long vectors
    if len(p) > 99:
        return 1 - distance.correlation(p,q)        
    
    p_mean = sum(p) / len(p)
    p_deviations = [(pi-p_mean) for pi in p]
    
    q_mean = sum(q) / len(q)
    q_deviations = [(qi-q_mean) for qi in q]
    
    cov = sum(pd * qd for pd,qd in zip(p_deviations, q_deviations))
        
    sds_product = math.sqrt(sum((pd)**2 for pd in p_deviations) * sum((qd)**2 for qd in q_deviations))
    
    if sds_product != 0:
        r = cov / sds_product
    else:
        r = 0
    return r

In [10]:
def calc_similarity(user2product, target, other_user, sim_measure, threshold=0):
    """This code extracts the set of shared rated items between two users, and computes their similarity. It is weighted
    by the amount of shared items, divided by the total amount of items rated by the target user."""
    # found some explanation here https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada
    
    shared = list(set(user2product[target].keys()).intersection(set(user2product[other_user].keys())))
    
    if len(shared) <= threshold:
        return 0
    
    target_ratings = [v for k,v in user2product[target].items() if k in shared]# for i in [target, other_user]]
    other_user_ratings = [v for k,v in user2product[other_user].items() if k in shared]
    
    weight = len(shared)/len(user2product[target])
    similarity = weight*sim_measure(target_ratings, other_user_ratings)
    
    return similarity

In [11]:
user2item = defaultdict(dict)
for reviewerID, asin, overall in X_train:
    user2item[reviewerID][asin] = overall

user2item_test = defaultdict(dict)
for reviewerID, asin, overall in X_test:
    user2item_test[reviewerID][asin] = overall


In [18]:
print("no. of users:", len(user2item))
print("no of reviews in user 1:", len(list(user2item.items())[0][1]))

no. of users: 4987
no of reviews in user 1: 16


In [11]:
item2user = defaultdict(dict)
for reviewerID, asin, overall in X_train:
    item2user[asin][reviewerID] = overall
    
print("no. of items:", len(item2user))

item2user_test = defaultdict(dict)
for reviewerID, asin, overall in X_test:
    item2user_test[asin][reviewerID] = overall

print("no. of items:", len(item2user_test))

no. of items: 3568
no. of items: 2609


In [21]:
#This function ranks the items within a dictionary of items and their ratings by users.
def rankids(useritemdict, thresh):
    rankedt = []
    rankedb = []
    scoredict = {}
    
    for k, v in useritemdict.items():
        itemdict = v
        for itemID, stars in itemdict.items():
            p = (stars, k[1])
            if itemID in scoredict:
                scoredict[itemID].append(p)
            else:
                scoredict[itemID] = [p]
    for ID, IDscortup in scoredict.items():
        starlist = []
        for scortup in IDscortup:
            starlist.append(scortup[0])
        a = np.mean(starlist)
        q = []
        for r in IDscortup:
            q.append(r[0] * r[1])
        IDandweighted = (ID, np.mean(q))
        if a > thresh:
            rankedt.append(IDandweighted)
        else:
            rankedb.append(IDandweighted)            
    
    rankedt.sort(key=itemgetter(1))
    rankedb.sort(key=itemgetter(1))
    rankedt.reverse()
    rankedb.reverse()
    return(rankedt, rankedb)

In [26]:
def recommend(userid, sim_measure, user2item, k, n):
    """This function takes a user id, similarity measure, the user2product dictionary, and k as input. It calculates the
    k most similar users. It outputs a dictionary, with as keys tuples containing the k similar users ID and their
    similarity, and as values a dictionary containing the ratings for all of their items"""
    reviewers = [user for user in user2item.keys() if user != userid]
    similarities = [(other_user, calc_similarity(user2item, userid, other_user, sim_measure)) for other_user in reviewers]
    k_similarities = sorted(similarities, key = lambda x: x[1], reverse=True)[:k]
    output = dict()
    for user_tup in k_similarities:
        output[user_tup] = user2item[user_tup[0]]
    
    lists = rankids(output, 2.49)
    #return(output)
    toplist = lists[0]
    botlist = lists[1]
    return (toplist + botlist)[:n]

In [32]:
#recommend('AZPWAXJG9OJXV', cosine_similarity, user2item, 50)

In [12]:
class kNN(object):
    """ k-Nearest Neighbour """
    
    def __init__(self, x2y: dict, sim_measure, name=None, k = 100, n=10, quiet = True):
        print("Building object...")
        self.x2y = x2y                     # For example, user2item matrix
        self.k = k
        self.n = n
        self.quiet = quiet
        self.sim_measure = sim_measure     # Like cosine similarity
        # The naming makes sure that you can load from a file
        # Assuming the sample set is not changed
        if name:
            self.similarities=pickle.load(open(f"data/{name}_similarities.pkl", 'rb'))
            self.neighborhood=pickle.load(open(f"data/{name}_neighboorhood.pkl", 'rb'))
        else:
            self.find_all_similarities()   # If the similarities are already stored
            self.get_neighborhoods()   # Likewise
        print("Done.")
    
    def find_all_similarities(self):
        """ Creates the similarity pairs for each item.
        If the similarities are already loaded, calculations can be skipped. """
        if not self.quiet: print("Making a similarity matrix...")
        sims = defaultdict(dict)
        for id1, id2 in combinations(self.x2y.keys(), 2):
            sims[id1][id2] = calc_similarity(self.x2y, id1, id2, self.sim_measure)
        self.similarities = sims
        
    def update_similarities(self, x2y_new):
        """ Rather than recalculating similarities, this function takes in the test
        set and adds the item similarities in that dictionary. """
        if not self.quiet: print("Updating...")
        for id1, id2 in product(x2y_new, self.x2y.keys()):
            self.similarities[id1][id2] = calc_similarity(self.x2y, id1, id2, self.sim_measure)
    
    def get_neighborhoods(self):
        """ For every item, it creates an item neighbourhood with similarity rates. 
        This is done via a default dictionary with values as dictionaries. """
        if not self.quiet: print(f"Setting up the size-{self.k} neighbourhoods...")
        self.neighborhood = dict()
        for x in self.similarities.keys():
            self.neighborhood[x] = dict(sorted(self.similarities[x].items(),
                                               key = itemgetter(1),
                                               reverse = True)[:self.k])
    
    def getPredictionsForItems(self, an_id, predict_item):
        """ Takes a user id and an item id, uses the neighborhood of the item
        to give a weighted prediction of what the user would rate the item"""
        weigthed_scores = list()
        similarities = list()
        for item, sim in self.neighborhood[predict_item].items():
            if an_id in self.x2y[item]:
                weigthed_scores.append(sim * self.x2y[item][an_id])
                similarities.append(sim)
        if not sum(similarities):
            return -1
        return float(f"{sum(weigthed_scores) / sum(similarities):.2f}")
    
    def recommend_item(self, an_id, reset = False):
        """ Gets the similarities, neighborhoods and the target vector
        Then gets predictions according to the vector. Finally recommends the top
        predictions."""
        if reset:
            self.find_all_similarities()
            self.get_neighborhoods()
        if not self.quiet: print("Recommending...")
        recommendations = dict()
        for item in self.neighborhood:
            recommendations[item] = self.getPredictionsForItems(an_id, item)
        return dict(sorted(list(recommendations.items()), key=lambda x:x[1], reverse=True)[:self.n])
    
    def save_state(self, name):
        """ To save the state to prevent recomputing each time"""
        file_stream = open(f'data/{name}_similarities.pkl', 'wb')
        file_stream2 = open(f'data/{name}_neighboorhood.pkl', 'wb')
        pickle.dump(self.similarities, file_stream)
        pickle.dump(self.neighborhood, file_stream2)
                          
    def load_state(self, name):
        """ To load the state to prevent recomputing each time"""
        file_stream = open(f'data/{name}_similarities.pkl', 'rb')
        file_stream2 = open(f'data/{name}_neighboorhood.pkl', 'rb')
        self.similarities = pickle.load(file_stream)
        self.neighborhood = pickle.load(file_stream2)
    
    def divide_test_set(self, x2y_new, ratio = 0.8):
        train_set = dict()
        test_set = dict()
        
        # Divide the
        if not self.quiet: print("Creating profiles...")
        keys = self.x2y.keys()
        for item, neigh in x2y_new.items():
            item_list = list(neigh.items())
            test_set[item] = dict(item_list[int(len(item_list)*ratio):])
            train_set[item] = dict(item_list[:int(len(item_list)*ratio)])
            if item in keys:
                self.x2y[item] = {**self.x2y[item], **train_set[item]}
        return train_set, test_set
    
    def evaluate(self, x2y_new, train_set=None, test_set=None, ratio = 0.8):
        """ Evaluates the model success via Mean Square Value. Ratio is the test to training
        split ratio of each item neighborhood. """
        if not train_set:
            train_set, test_set = self.divide_test_set(x2y_new, ratio)
        
        # Enter the new profiles to the neighborhoods
        if not self.quiet: print("Updating...")
        self.update_similarities(train_set)
        self.get_neighborhoods()
        
        if not self.quiet: print("Evaluating...")
        costs = dict()
        missed_item = 0
        # Iterate over every item to be predicted
        for test_item, _ in x2y_new.items():
            costs[test_item] = 0
            missed_person = 0
            # Iterate over every person in the test set that haven't been added to memory
            for test_person, rate in test_set[test_item].items():
                predicted = self.getPredictionsForItems(test_person, test_item)
                # Predicted will be -1 if no prediction can be made, the length is adjusted
                if predicted is -1:
                    missed_person -= 1
                costs[test_item] += (rate-predicted)**2
            # Total length also being ajusted if no prediction about the item could be made
            if len(test_set[test_item]) + missed_person is 0:
                costs[test_item] = 0
                missed_item -= 1
            else:
                costs[test_item] = costs[test_item]/(len(test_set[test_item]) + missed_person)
        # If all predictions fail, return 30, above the possible cost
        if len(x2y_new)+ missed_item is 0:
            return 30
        else:
            return sum(costs.values())/(len(x2y_new) + missed_item)
    
    def optimize(self, x2y_train, x2y_test, measures, k_range):
        """ Optimizer takes a training, a test and a measures dictionary.
        Also, it takes either a range or a list of k values. Then it experiments
        with all of them to find the optimal values."""
        
        # Setting the variables, division of test set to profiling and predicting
        self.x2y = x2y_train
        self.find_all_similarities()
        test_training, test_testing = self.divide_test_set(x2y_test)
        
        # Testing of each similarity measure with default k
        costs = dict()
        for name, measure in measures.items():
            self.sim_measure = measure
            costs[name] = self.evaluate(x2y_test, train_set=test_training, test_set=test_testing)
        optimum_mes = min(list(costs.items()), key=lambda x:x[1])
        
        print(f"Optimum measure is: {optimum_mes[0]} with cost {costs[optimum_mes[0]]}")
        self.sim_measure = measures[optimum_mes[0]]
        
        # Testing of each k value with the optimum measure
        costs = dict()
        for temp_k in k_range:
            self.k = temp_k
            costs[temp_k] = self.evaluate(x2y_test, train_set=test_training, test_set=test_testing)
            print(f"With K: {temp_k} cost is: {costs[temp_k]}")
        optimum_k = min(list(costs.items()), key=lambda x:x[1])[0]
        
        # Getting the optimum combination and testing it
        print(f"Optimum K value is: {optimum_k} with cost {costs[optimum_k]}")
        self.k = optimum_k
        cost = self.evaluate(x2y_test, train_set=test_training, test_set=test_testing)
        
        return [optimum_mes[0], optimum_k, cost]
 

In [14]:
# This code initializes and optimizes the object
measure2function = {"euclidean" : euclidean_similarity, "cosine": cosine_similarity, "pearson": pearson_correlation}
aKNN = kNN(item2user, cosine_similarity)
aKNN.optimize(item2user, item2user_test, measure2function, [5, 10, 20, 50, 100])

Building object...
Done.
Optimum measure is: cosine with cost 3.523769810901001
With K: 5 cost is: 7.056626445264453
With K: 10 cost is: 6.091338586309523
With K: 20 cost is: 5.193909600642929
With K: 50 cost is: 4.093544473197786
With K: 100 cost is: 3.523769810901001
Optimum K value is: 100 with cost 3.523769810901001


['cosine', 100, 3.523769810901001]

In [61]:
bKNN = kNN(item2user, pearson_correlation, name = 'item2user')
recs = bKNN.recommend_item('A22IK3I6U76GX0')
# This can be all zeroes, if the user happens to be in the test set. Then change the name

# for ID, rate in recs.items():
#     print(f"Item {ID} is estimated to be rated: {rate}")
#print("Total evaluation:", aKNN.evaluate(user2item_test))
bKNN.evaluate(user2item_test)

Making a similarity matrix...
Setting up the 10 neighbourhoods...
Recommending...
Making a similarity matrix...
Setting up the 10 neighbourhoods...
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 4.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 4.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 4.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 4.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 4.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 3.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 5.0
Predicted: 0
Given: 4.0
P

In [76]:
def recommend(userid, sim_measure, user2item, k, n):
    """This function takes a user id, similarity measure, the user2product dictionary, and k as input. It calculates the
    k most similar users. It outputs a dictionary, with as keys tuples containing the k similar users ID and their
    similarity, and as values a dictionary containing the ratings for all of their items"""
    reviewers = [user for user in user2item.keys() if user != userid]
    similarities = [(other_user, calc_similarity(user2item, userid, other_user, sim_measure)) for other_user in reviewers]
    k_similarities = sorted(similarities, key = lambda x: x[1], reverse=True)[:k]
    output = dict()
    for user_tup in k_similarities:
        output[user_tup] = user2item[user_tup[0]]
    
    lists = rankids(output, 2.49)
    #return(output)
    toplist = lists[0]
    botlist = lists[1]
    return (toplist + botlist)[:n]
    

#Evaluation function, which compares items within test users and
#the recommendations for that user, based on a fraction of their purchase data.
def evaluate2(x2y,k,n,measure, testdata, ratio):
    accuracies =[]
    for user,group in testdata.items():
        reckeys = recommend(user, measure, x2y, k,n)
        overlap = 0
        #print(overlap)
        itemscount = 0
        iter = 0
        lim = math.trunc(len(group) * ratio)
        for item,score in reckeys:
            iter += 1
            itemscount += 1
            if iter == lim:
                break
            elif item in group.keys():
                overlap += 2
            else:
                pass
        accuracy = overlap / (len(reckeys)+ itemscount)
        accuracies.append(accuracy)
    eval2 = np.mean(accuracies)  
    return eval2
    
    
print(evaluate2(user2item,10,10,cosine_similarity,user2item_test, 0.3))
print(evaluate2(user2item,10,10,cosine_similarity,user2item_test, 0.5))
print(evaluate2(user2item,10,10,cosine_similarity,user2item_test, 0.7))
print(evaluate2(user2item,10,10,cosine_similarity,user2item_test, 0.9))

0.0029870515270812572
0.0043760251972968945
0.004701969652065214
0.004958882660037224


In [ ]:
def optimiser(k_list, n_list, sim_measures, ratio, test_data):
    scores = []
    for k in k_list:
        for n in n_list:
            for measure in sim_measures:
                score = evaluate(x2y, k, n, measure, test_data, ratio)
                string = ''.join('k=',k,', n=',n,', measure=',measure,', ratio=',ratio)
                scores.append((string, score))